In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.auto import tqdm

In [ ]:
q1train = pd.read_csv('train.csv', usecols=['question1'])
q2train = pd.read_csv('train.csv', usecols=['question2'])

q1test = pd.read_csv('test.csv', usecols=['question1'])
q2test = pd.read_csv('test.csv', usecols=['question2'])

In [ ]:
q1train.columns = ['question']
q2train.columns = ['question']
q1test.columns = ['question']
q2test.columns = ['question']

In [ ]:
questions = pd.concat([q1train, q2train, q1test, q2test]).reset_index(drop=True)

In [ ]:
import re

def clean(x):
    x = re.sub(r'\W', ' ', str(x))
    x = re.sub(r'\s+', ' ', str(x))
    return str(x)

In [ ]:
questions['question'] = questions['question'].apply(clean)
questions['len'] = questions['question'].apply(lambda x : len(str(x).split()))

In [ ]:
print(len(questions))
questions.drop_duplicates(subset=['question'], inplace=True)
questions.drop(questions[questions['len'] <= 1].index, inplace=True)
questions.dropna(inplace=True)
print(len(questions))

In [ ]:
questions = questions['question'].values

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [ ]:
questions = [TaggedDocument(words=word_tokenize(q.lower()), tags=[str(i)]) for i, q in tqdm(enumerate(questions))]

In [ ]:
max_epochs = 7
vec_size = 128
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(questions)
print('Model Trainign Started')

for epoch in tqdm(range(max_epochs)):
    model.train(questions,
                total_examples=model.corpus_count,
                epochs=model.iter)

    model.alpha -= 0.0002
    model.min_alpha = model.alpha

    model.save("d2v.model")
    print("Model Saved")